# TensorBoard Scalars: Logging training metrics in Keras

## Overview

机器学习总是涉及了解关键指标，如损失以及它们如何随着训练的进展而变化。例如，这些指标可以帮助您了解您是否过拟合，或者您是否进行了不必要的训练。您可能希望在不同的训练运行中比较这些指标，以帮助调试和改进您的模型。

TensorBoard的 **Scalars Dashboard** 允许您毫不费力地使用简单的API可视化这些指标。本教程提供了非常基本的示例，以帮助您了解如何在开发 Keras 模型时将这些API与TensorBoard一起使用。您将学习如何使用Keras TensorBoard回调和TensorFlow Summary API来可视化默认和自定义标量。

## Setup

In [1]:
%load_ext tensorboard

In [2]:
from datetime import datetime
from packaging import version

import tensorflow as tf
from tensorflow import keras

import numpy as np

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

TensorFlow version:  2.6.0


## Set up data for a simple regression

您现在将使用Keras来计算回归，即找到配对数据集的最佳拟合线。（虽然使用神经网络和梯度下降对此类问题来说太过分了，但它确实是一个非常容易理解的例子。）

您将使用TensorBoard来观察训练和测试 **损失** 在各个 epoch 的变化。希望随着时间的推移，您会看到训练和测试损失减少，然后保持稳定。

首先，大致沿着y = 0.5x + 2线生成1000个数据点。将这些数据点拆分为训练和测试集。你的希望是神经网络学会这种关系。

In [3]:
data_size = 1000
# 80% of the data is for training.
train_pct = 0.8

train_size = int(data_size * train_pct)

# Create some input data between -1 and 1 and randomize it.
x = np.linspace(-1, 1, data_size)
np.random.shuffle(x)

# Generate the output data.
# y = 0.5x + 2 + noise
y = 0.5 * x + 2 + np.random.normal(0, 0.05, (data_size, ))

# Split into test and train pairs.
x_train, y_train = x[:train_size], y[:train_size]
x_test, y_test = x[train_size:], y[train_size:]

## Training the model and logging loss

您现在可以定义、训练和评估您的模型了。

要在训练时记录损失标量，您需要执行以下操作：

- 创建 Keras TensorBoard 回调
- 指定日志目录
- 将 TensorBoard 回调传递给 Keras 的 Model.fit()。

TensorBoard从日志目录层次结构中读取日志数据。在本笔记本中，根日志目录是 `logs/scalars`，后缀为时间戳子目录。时间戳子目录使您可以在使用TensorBoard和迭代模型时轻松识别和选择训练运行。

In [4]:
logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

model = keras.models.Sequential([
    keras.layers.Dense(16, input_dim=1),
    keras.layers.Dense(1),
])

model.compile(
    loss='mse', # keras.losses.mean_squared_error
    optimizer=keras.optimizers.SGD(learning_rate=0.2),
)

print("Training ... With default parameters, this takes less than 10 seconds.")
training_history = model.fit(
    x_train, # input
    y_train, # output
    batch_size=train_size,
    verbose=0, # Suppress chatty output; use Tensorboard instead
    epochs=100,
    validation_data=(x_test, y_test),
    callbacks=[tensorboard_callback],
)

print("Average test loss: ", np.average(training_history.history['loss']))

2022-04-11 18:40:00.546609: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-04-11 18:40:00.546620: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2022-04-11 18:40:00.546912: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2022-04-11 18:40:00.550266: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-04-11 18:40:00.550407: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1
Training ... With default parameters, this takes less than 10 seconds.


2022-04-11 18:40:01.187780: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-04-11 18:40:01.187964: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-04-11 18:40:01.255673: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-04-11 18:40:01.651416: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-04-11 18:40:01.679157: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-04-11 18:40:01.679166: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2022-04-11 18:40:01.681821: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2022-04-11 18:40:01.685197: I tensorflow/core/profiler/lib/profiler_ses

Average test loss:  0.044937716908752916


## Examining loss using TensorBoard

现在，启动TensorBoard，指定您上面使用的根日志目录。

等待几秒钟，让TensorBoard的用户界面旋转。

In [5]:
%tensorboard --logdir logs/scalars

您可能会看到TensorBoard显示消息“No dashboards are active for the current data set”。这是因为初始日志数据尚未保存。随着训练的进行，Keras模型将开始记录数据。TensorBoard将定期刷新并显示您的标量指标。如果您不耐烦，您可以点击右上角的刷新箭头。

当您观看训练进度时，请注意训练和验证损失如何迅速减少，然后保持稳定。事实上，你可以在25个 epoch 后停止训练，因为在那之后训练没有太大改善。

将鼠标悬停在图表上以查看特定数据点。您还可以尝试使用鼠标放大，或选择其中的一部分以查看更多详细信息。

注意左侧的 “Runs” 选择器。“run” 表示一轮训练的一组日志，在这种情况下是 `Model.fit()` 的结果。随着时间的推移，开发人员在实验和开发模型时通常会运行很多很多。

使用 Runs 选择器选择特定 runs，或仅从训练或验证中进行选择。比较 runs 将帮助您评估哪个版本的代码可以更好地解决问题。

好的，TensorBoard的损失图表明，在训练和验证中，损失持续下降，然后稳定下来。这意味着该模型的指标可能非常好！现在看看模型在现实生活中的表现。

给定输入数据（60、25、2），线y = 0.5x + 2应该产生（32, 14.5, 3）。模型是否一致？

In [6]:
print(model.predict([60, 25, 2]))

[[31.797571]
 [14.415272]
 [ 2.992618]]


2022-04-11 18:45:45.627789: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


## Logging custom scalars

如果您想记录自定义值，例如动态学习率，该怎么办？为此，您需要使用TensorFlow Summary API。

重新训练回归模型并记录自定义学习率。方法如下：

- 使用 `tf.summary.create_file_writer()` 创建一个 file writer。
- 定义自定义学习速率函数。这将传递给 Keras `LearningRateScheduler` 回调。
- 在学习率函数中，使用 `tf.summary.scalar()` 记录自定义学习率。
- 将 `LearningRateScheduler` 回调传递给 `Model.fit()`。

一般来说，要记录自定义标量，您需要将 `tf.summary.scalar()` 与 file writer 一起使用。file writer 负责将此运行的数据写入指定目录，并在您使用 `tf.summary.scalar()` 时隐式使用。

In [7]:
logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
file_writer = tf.summary.create_file_writer(logdir + "/metrics")
file_writer.set_as_default()

def lr_schedule(epoch):
    learning_rate = 0.2
    if epoch > 10:
        learning_rate = 0.02
    if epoch > 20:
        learning_rate = 0.01
    if epoch > 50:
        learning_rate = 0.005
        
    tf.summary.scalar('learning rate', data=learning_rate, step=epoch)
    return learning_rate

lr_callback = keras.callbacks.LearningRateScheduler(lr_schedule)
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

model = keras.models.Sequential([
    keras.layers.Dense(16, input_dim=1),
    keras.layers.Dense(1),
])

model.compile(
    loss='mse', # keras.losses.mean_squared_error
    optimizer=keras.optimizers.SGD(),
)

training_history = model.fit(
    x_train, # input
    y_train, # output
    batch_size=train_size,
    verbose=0, # Suppress chatty output; use Tensorboard instead
    epochs=100,
    validation_data=(x_test, y_test),
    callbacks=[tensorboard_callback, lr_callback],
)

2022-04-11 18:53:52.065912: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-04-11 18:53:52.065933: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2022-04-11 18:53:52.066018: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2022-04-11 18:53:52.163229: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-04-11 18:53:52.223709: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-04-11 18:53:52.247169: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-04-11 18:53:52.247178: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2022-04-11 18:53:52.256945: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2022-04-11 

让我们再看看TensorBoard。

In [9]:
%tensorboard --logdir logs/scalars

Reusing TensorBoard on port 6006 (pid 76241), started 0:13:49 ago. (Use '!kill 76241' to kill it.)

使用左侧的“Runs”选择器，请注意您运行了 `<timestamp>/metrics` 。选择此运行会显示一个“学习率”图表，允许您验证此运行期间学习率的进度。

您还可以将此运行的训练和验证损失曲线与之前的运行进行比较。您还可能会注意到，学习率时间表根据 epoch 返回了离散值，但学习率图可能看起来很流畅。TensorBoard有一个平滑参数，您可能需要将该参数调低到零才能看到未平滑的值。

In [10]:
print(model.predict([60, 25, 2]))

[[31.797157 ]
 [14.415102 ]
 [ 2.9926066]]


2022-04-11 18:55:53.074869: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
